In [11]:
!pip install transformers datasets
!pip install tqdm

In [1]:
from google.colab import files
uploaded = files.upload()

Saving input.txt to input.txt


In [2]:
from datasets import Dataset
from transformers import GPT2Tokenizer
import pandas as pd
import io

# Load the text data from the uploaded file
file_content = uploaded['input.txt'].decode('utf-8')
df = pd.DataFrame({'text': [file_content]})

# Convert the pandas DataFrame to a Dataset object
dataset = Dataset.from_pandas(df)

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    return tokenizer(batch['text'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [ ]:
from datasets import Dataset

# Step 1: Read raw lines from the file
with open("input.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()

# Step 2: Clean the lines (remove empty ones and strip whitespace)
lines = [line.strip() for line in lines if line.strip() != ""]

# Step 3: Wrap into Hugging Face dataset format
dataset = Dataset.from_dict({"text": lines})

# Preview the first few entries
dataset[:5]


In [ ]:

import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
def tokenize(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)
    tokens["labels"] = tokens["input_ids"].copy()  # Needed for training loss
    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True)


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    run_name="gpt2-test-run",
    report_to="none",
    num_train_epochs=1,
    max_steps=100,
    per_device_train_batch_size=4,
    save_steps=50,
    save_total_limit=1,
    logging_dir="./logs",
    logging_steps=10
)



In [ ]:
small_dataset = tokenized_dataset.select(range(1500))  # use first 1000 samples


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments

# Load model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_dataset,
)
trainer.train()

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

prompt = "Hello"
output = generator(prompt, max_length=50, num_return_sequences=1, do_sample=True)

print("📝 Generated Text:\n")
print(output[0]['generated_text'])
